<h1>Federated Learning using MNIST</h1>

<h2>Centralized Training</h2>

In [1]:
import tensorflow as tf

# Import Datasets
mnist = tf.keras.datasets.mnist
(data_train, label_train), (data_test, label_test) = mnist.load_data()

# Data pre-processing
data_train, data_test = data_train/255.0, data_test/255.0

# Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])
    
# Compile model
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# Train Model
model.fit(data_train, label_train, epochs=5)

# Get results
test_loss, test_acc = model.evaluate(data_test, label_test)
print("test loss: ", test_loss)
print("test accuracy: ", test_acc)

Metal device set to: Apple M1 Max
Epoch 1/5


2022-03-27 01:17:33.520198: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-27 01:17:33.520304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-27 01:17:33.648937: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  25/1875 [..............................] - ETA: 8s - loss: 1.2110 - accuracy: 0.6600

2022-03-27 01:17:33.776840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.2032 - accuracy: 0.9403
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0804 - accuracy: 0.9760
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0523 - accuracy: 0.9830
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0371 - accuracy: 0.9883
Epoch 5/5
 40/313 [==>...........................] - ETA: 1s - loss: 0.1044 - accuracy: 0.9688

2022-03-27 01:18:13.323594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 4ms/step - loss: 0.0814 - accuracy: 0.9763
test loss:  0.08135548233985901
test accuracy:  0.976300060749054


<h2>Test "get_weights", "set_weights"</h2>

In [2]:
weights = model.get_weights()

cp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

cp_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

cp_model.set_weights(weights)

# Get results
test_loss, test_acc = model.evaluate(data_test, label_test)
print("test loss: ", test_loss)
print("test accuracy: ", test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 0.0814 - accuracy: 0.9763
test loss:  0.08135548233985901
test accuracy:  0.976300060749054


<h2>Data Split 3 of Clients</h2>

In [3]:
c1_data_train = data_train[:20000, :, :]
c1_label_train = label_train[:20000]

c2_data_train = data_train[20000:40000, :, :]
c2_label_train = label_train[20000:40000]

c3_data_train = data_train[40000:60000, :, :]
c3_label_train = label_train[40000:]

print(c1_data_train.shape, len(c1_label_train))
print(c2_data_train.shape, len(c2_label_train))
print(c3_data_train.shape, len(c3_label_train))

(20000, 28, 28) 20000
(20000, 28, 28) 20000
(20000, 28, 28) 20000


<h2>Get Weights from Client 1</h2>

In [4]:
# Train Client 1
c1_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c1_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c1_model.fit(c1_data_train, c1_label_train, epochs=5)

c1_weights = c1_model.get_weights()

Epoch 1/5
  7/625 [..............................] - ETA: 10s - loss: 1.9408 - accuracy: 0.4464

2022-03-27 01:18:16.111648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3158 - accuracy: 0.9082
Epoch 2/5
625/625 [==============================] - 3s 4ms/step - loss: 0.1310 - accuracy: 0.9607
Epoch 3/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0748 - accuracy: 0.9787
Epoch 4/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0505 - accuracy: 0.9857
Epoch 5/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0343 - accuracy: 0.9902


<h2>Get Weights from Client 2</h2>

In [5]:
# Train Client 2
c2_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c2_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c2_model.fit(c2_data_train, c2_label_train, epochs=5)

c2_weights = c2_model.get_weights()

Epoch 1/5
 24/625 [>.............................] - ETA: 2s - loss: 1.2180 - accuracy: 0.6732

2022-03-27 01:18:29.588050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3203 - accuracy: 0.9067
Epoch 2/5
625/625 [==============================] - 3s 4ms/step - loss: 0.1338 - accuracy: 0.9598
Epoch 3/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0833 - accuracy: 0.9748
Epoch 4/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0539 - accuracy: 0.9841
Epoch 5/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0373 - accuracy: 0.9889


<h2>Get Weights from Client 3</h2>

In [6]:
# Train Client 3
c3_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c3_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c3_model.fit(c3_data_train, c3_label_train, epochs=5)

c3_weights = c3_model.get_weights()

Epoch 1/5
 25/625 [>.............................] - ETA: 2s - loss: 1.2495 - accuracy: 0.6413

2022-03-27 01:18:42.974892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3136 - accuracy: 0.9077
Epoch 2/5
625/625 [==============================] - 3s 4ms/step - loss: 0.1377 - accuracy: 0.9603
Epoch 3/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0818 - accuracy: 0.9760
Epoch 4/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0533 - accuracy: 0.9846
Epoch 5/5
625/625 [==============================] - 3s 4ms/step - loss: 0.0375 - accuracy: 0.9885


<h2>Aggregation Weights</h2>

In [8]:
import numpy as np

agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0
# agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0

agg_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

agg_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

agg_model.set_weights(agg_weights)

# Get results
test_loss, test_acc = agg_model.evaluate(data_test, label_test)
print("test loss: ", test_loss)
print("test accuracy: ", test_acc)

 13/313 [>.............................] - ETA: 1s - loss: 0.0514 - accuracy: 0.9832 

/var/folders/86/dxn6gj7x5jdcw5893s56tz2h0000gn/T/ipykernel_97547/4010025759.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0
2022-03-27 01:18:56.326087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 4ms/step - loss: 0.0839 - accuracy: 0.9737
test loss:  0.08392113447189331
test accuracy:  0.9737000465393066
